In [22]:
import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

from numpy import *

#from statistics import *

import plotly.express as px

import plotly.graph_objects as go

import seaborn as sns

from wordcloud import WordCloud, STOPWORDS

#import dfply

#from dfply import *

#import pandas as pd

#from plotnine import *

#from plotnine.data import *


                                                    Read in data

In [2]:

EssayFilePath = "C:/Users/mbles/Desktop/Vaccines are Beneficial.txt"

ResumeFilePath = "C:/Users/mbles/Desktop/Resume.txt"

LabFilePath = "C:/Users/mbles/Desktop/EEE 334 Lab 1.txt"


EssayLines = ""

Resumelines = ""

LabLines = ""


def Read(FilePath, lines):
    with open(FilePath, "r", encoding = 'utf-8') as File:
        lines = File.read()
    return(lines)

try:    
    EssayText = Read(EssayFilePath, EssayLines)
except:
    EssayFilePath = input("Please enter the file path for the essay.\n")
    EssayText = Read(EssayFilePath, EssayLines)

try:
    ResumeText = Read(ResumeFilePath, Resumelines)
except:
    ResumeFilePath = input("\nPlease enter the file path for the resume.\n")
    ResumeText = Read(ResumeFilePath, Resumelines)
    
try:    
    LabText = Read(LabFilePath, LabLines)
except:
    LabFilePath = input("\nPlease enter the file path for the lab report.\n")
    

                                                Clean  and Sort Data

In [14]:
non_letter_characters = ['[','@','_','!','#','$','%','^','&','*',
                      '(',')','<','>','?','/', '}','{','~',':',']',
                         ',','0', '1','2','3','4','5','6','7','8',
                         '9','\t','\n', ' ', '"', "'", '-','.',';',
                         '=','—','-','–','‘','“','”','\ufeff','…','’',
                         '\\\\', '\\', " \ ", ' n ', '  ','    ','   ',
                         '  n ', '⋅', '→','|', '〖', '〗','√','ω','μ','µa',
                         'π','+','δ','°','≈','\u2008','\u2008f','\u2061',
                         '∫','▒','±','●','δ','ω', 'ω	', 'µ', 'δ']

def CleanData(Text):
    for i in non_letter_characters:
        Text = Text.replace(i,' ')
    
    Text = Text.lower()
    
    return(Text)

EssayText = CleanData(EssayText)

ResumeText = CleanData(ResumeText)

LabText = CleanData(LabText) 

                                                 Get Letter Frequencies  

In [23]:
def GetFrequencies(Text):  
    LetterFrequencies = {}
    for keys in Text:
        LetterFrequencies[keys] = LetterFrequencies.get(keys, 0) + 1
    return(LetterFrequencies)

EssayFrequencyDict = GetFrequencies(EssayText)

ResumeFrequencyDict = GetFrequencies(ResumeText)

LabFrequencyDict = GetFrequencies(LabText)

                                                Convert to Data Frames and Sort

In [24]:
def ConvertAndSort(FrequencyDict, column2):
    Frequency_df = pd.DataFrame(FrequencyDict.items(), columns=['Letter', column2]) 

    Frequency_df = Frequency_df.sort_values("Letter")

    Frequency_df = Frequency_df[Frequency_df.Letter != " "].reset_index()  #renumber rows from 0 to 1 from the top down
    
    Frequency_df.pop('index')  #remove column containing prior order as I have no need of it
    
    return(Frequency_df)
    
EssayFrequency_df = ConvertAndSort(EssayFrequencyDict, 'Essay Frequency')

ResumeFrequency_df = ConvertAndSort(ResumeFrequencyDict, 'Resume Frequency')

LabFrequency_df = ConvertAndSort(LabFrequencyDict, 'Lab Frequency')


                                                               Merge Dataframes

In [19]:
#"Letter" columnin each df is the same, so two of them are being concantenated without it

temp = pd.concat([EssayFrequency_df, ResumeFrequency_df.drop(['Letter'], axis = 1, inplace = False)], axis = 1, join = 'outer')

merged_data = pd.concat([temp, LabFrequency_df.drop(['Letter'], axis = 1, inplace = False)], axis = 1, join = 'outer')

merged_data.head()

,Letter,Essay Frequency,Resume Frequency,Lab Frequency
0,a,1223,233,996
1,b,163,37,187
2,c,650,185,448
3,d,503,108,448
4,e,1901,300,1471


                                                         Get Summary Stats

In [28]:
SummaryStats = (merged_data.describe()).round(decimals = 2)

SummaryStats

print(SummaryStats)

GroupByLetter = merged_data.transpose()

GroupByLetter.columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 
                       'h', 'i', 'j', 'k', 'l', 'm', 'n', 
                       'o', 'p', 'q', 'r', 's', 't', 'u', 
                       'v', 'w', 'x', 'y', 'z']

GroupByLetter = GroupByLetter.drop('Letter')

SummaryStatsByLetter = GroupByLetter.describe().round(decimals = 2)

print(SummaryStatsByLetter)

       Essay Frequency  Resume Frequency  Lab Frequency
count            26.00             26.00          26.00
mean            530.85            110.31         479.31
std             477.23             95.49         392.24
min               7.00              5.00          14.00
25%             189.00             27.25         197.00
50%             332.00             93.00         375.00
75%             864.25            185.00         809.00
max            1901.00            300.00        1471.00
           a    b    c    d     e    f    g    h    i  j  ...  q    r    s  \
count      3    3    3    3     3    3    3    3    3  3  ...  3    3    3   
unique     3    3    3    3     3    3    3    3    3  3  ...  3    3    3   
top     1223  163  650  503  1901  308  207  623  992  8  ...  7  724  944   
freq       1    1    1    1     1    1    1    1    1  1  ...  1    1    1   

           t    u    v    w   x    y   z  
count      3    3    3    3   3    3   3  
unique     3    3  

                                                              Boxplots

In [29]:
px.box(merged_data).show()

px.box(GroupByLetter).show()

ValueError: Plotly Express cannot process wide-form data with columns of different type.

                                                             Bar Graphs

In [ ]:
%matplotlib inline

def BarGraph(df, Title, Color, Y):
    bar_graph = df.plot.bar(x = "Letter", y = Y, rot = 70, title = Title, color = Color, legend = False)

BarGraph(EssayFrequency_df, "Frequencies of Letters in Essay", "purple", 'Essay Frequency')

BarGraph(ResumeFrequency_df, "Frequencies of Letters in Resume", "pink", 'Resume Frequency')

BarGraph(LabFrequency_df, "Frequencies of Letters in Lab Report", "blue", 'Lab Frequency')


In [ ]:
%matplotlib inline

sns.set_palette("hls", 26)

def histogram(df, Title, X):
    hist = sns.displot(df, x = X, bins = 15, hue = 'Letter', multiple = 'stack', stat = 'probability')
    hist.fig.suptitle(Title)
    
histogram(EssayFrequency_df, "Frequencies of Letters in Essay", 'Essay Frequency')
histogram(ResumeFrequency_df, "Frequencies of Letters in Resume", 'Resume Frequency')
histogram(LabFrequency_df, "Frequencies of Letters in Lab Report", 'Lab Frequency')



In [ ]:

def CreateWordCloud(text):
    #Primary Source for writing this function: https://www.geeksforgeeks.org/generating-word-cloud-python/ 
    
    stopwords = set(STOPWORDS)

    # create wordcloud object
    wordcloud = WordCloud(background_color="white", max_words = 200, stopwords = stopwords)
    wordcloud.generate(text)

    plt.figure(figsize = (10, 10), facecolor = None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad = 0)

    plt.show()
    
CreateWordCloud(EssayText)
CreateWordCloud(ResumeText)
CreateWordCloud(LabText)
